In [1]:
import numpy as np
import pickle as pk
import os

hd_part1 = '/home/martin/container/hail/detectron2/products/hparam_tuning/run-3/hail_20210620_r1/all_images/hail_sizes/part1/hail_dict.pkl'
hd_part2 = '/home/martin/container/hail/detectron2/products/hparam_tuning/run-3/hail_20210620_r1/all_images/hail_sizes/part2/hail_dict.pkl'
hd_part3 = '/home/martin/container/hail/detectron2/products/hparam_tuning/run-3/hail_20210620_r1/all_images/hail_sizes/part3/hail_dict.pkl'

#List of indexes which are no hail:
no_hail_part1 = [3751] # leaves etc.
no_hail_part2 = [2939, 3250]

with open(hd_part1,'rb') as f:
    hd_p1 = pk.load(f)

for ind in no_hail_part1:
    hd_p1['major_ma'][ind] = np.nan
    hd_p1['minor_ma'][ind] = np.nan
    hd_p1['L_center'][ind] = np.nan    
    
with open(hd_part2,'rb') as f:
    hd_p2 = pk.load(f)

for ind in no_hail_part2:
    hd_p2['major_ma'][ind] = np.nan
    hd_p2['minor_ma'][ind] = np.nan
    hd_p2['L_center'][ind] = np.nan    
    
with open(hd_part3,'rb') as f:
    hd_p3 = pk.load(f)
    
ds = [hd_p1, hd_p2, hd_p3]
d = {}
for k in ['L_center', 'major_ma', 'minor_ma','hail_co','tile_names','img']:
    d[k] = tuple(d[k] for d in ds)

L_center = np.hstack(np.asarray(d['L_center'][:]))
names = np.hstack(np.asarray(d['tile_names'][:]))

major_ax = np.hstack(np.asarray(d['major_ma'][:]))
minor_ax =  np.hstack(np.asarray(d['minor_ma'][:]))
hail_co_x = np.hstack([d['hail_co'][0][0],d['hail_co'][1][0],d['hail_co'][2][0]])
hail_co_y = np.hstack([d['hail_co'][0][1],d['hail_co'][1][1],d['hail_co'][2][1]])

/tmp/ipykernel_6621/3817054109.py:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  L_center = np.hstack(np.asarray(d['L_center'][:]))
/tmp/ipykernel_6621/3817054109.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  names = np.hstack(np.asarray(d['tile_names'][:]))
/tmp/ipykernel_6621/3817054109.py:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creat

In [5]:
np.sum(~np.isnan(major_ax))

18209

In [66]:
import pandas as pd

data={'major_ax': major_ax, 'minor_ax': minor_ax, 'x_cord': hail_co_x, 'y_cord': hail_co_y}

df=pd.DataFrame(data)
df.index.name = 'hail_index'

df.to_csv('/home/martin/container/hail/jerome/hail_20210620_r1.csv', sep=',')

In [79]:
#Get number of transparent pixels in orthophoto 
stream = os.popen('convert /home/martin/container/hail/jerome/odm_orthophoto.png -alpha extract -negate -format "%[fx:mean*w*h]" info:')
output = stream.read()
print(output)

all_px = 5.72271*1e8
rel_px = all_px - np.float(output)
area = rel_px*1.5/1e6
print('Area: ', area)

7.04888e+07
Area:  752.6733


In [93]:
import PIL
from PIL import Image
#Load orthophoto image   
orthophoto_file = '/home/martin/container/autofs/data/hail/hail_20210620_r3/odm_orthophoto/odm_20210620_r3.png'
PIL.Image.MAX_IMAGE_PIXELS = 8046500000
image = np.asarray(Image.open(orthophoto_file))

sought = [0, 0, 0]
black = np.count_nonzero(np.all(image[:,:,0:3] == sought, axis = 2))
print("black: ", black)

rel_px = np.size(image,0)*np.size(image,1) - black
area = rel_px*1.5/1e6
print('Area: ', area)

black:  60310856


In [98]:
import PIL
from PIL import Image
#Load orthophoto image   
orthophoto_file = '/home/martin/container/autofs/data/hail/hail_20210620_r5/odm_orthophoto/odm_20210620_r5.png'
PIL.Image.MAX_IMAGE_PIXELS = 8046500000
image = np.asarray(Image.open(orthophoto_file))

sought = [0, 0, 0]
black = np.count_nonzero(np.all(image[:,:,0:3] == sought, axis = 2))
print("black: ", black)

rel_px = np.size(image,0)*np.size(image,1) - black
area = rel_px*1.5/1e6
print('Area: ', area)

black:  115232530
Area:  602.651205


In [49]:
#Miscellanus test
#Write to netcdf file
import netCDF4 as nc
import numpy as np

path_nc = '/home/martin/container/hail/jerome/'
if not os.path.exists(path_nc):
    os.makedirs(path_nc)
        
fn = path_nc+'hail_20210620_r1.nc'
ds = nc.Dataset(fn, 'w', format='NETCDF4')

major_ax_nc = ds.createVariable('major_ax', 'f4')
major_ax_nc.units = 'mm'

minor_ax_nc = ds.createVariable('minor_ax', 'f4')
minor_ax_nc.units = 'mm'

hail_x = ds.createVariable('hail_x', 'int')
hail_x.units = 'pixel'
hail_y = ds.createVariable('hail_y', 'int')
hail_y.units = 'pixel'

major_ax_nc = major_ax
minor_ax_nc = minor_ax
hail_x = hail_co_x
hail_y = hail_co_y

ds.close()